# MotorHandPro - Getting Started

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/01_getting_started.ipynb)

**Patent Pending:** U.S. Provisional Patent Application No. 63/842,846

This notebook provides an interactive introduction to MotorHandPro's Primal Logic framework.

## What You'll Learn

1. Core Primal Logic constants (D, I3, S)
2. Exponential memory weighting with λ (Lightfoot constant)
3. Control law implementation
4. Interactive visualization of control dynamics
5. Parameter tuning with immediate feedback

## Setup

First, let's install dependencies and clone the repository (if running on Colab).

In [ ]:
import sys
import os

# Check if running on Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab - installing dependencies...")
    !pip install -q numpy scipy matplotlib plotly pandas
    
    # Clone repo if not already present
    if not os.path.exists('/content/MotorHandPro'):
        !git clone https://github.com/STLNFTART/MotorHandPro.git /content/MotorHandPro
        os.chdir('/content/MotorHandPro')
    else:
        os.chdir('/content/MotorHandPro')
    
    # Install requirements
    !pip install -q -r requirements.txt
else:
    print("Running locally or on Brev")

print("✓ Setup complete!")

## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries imported")

## Core Primal Logic Constants

These constants emerge from the Planck tail cutoff calculation and form the foundation of the Primal Logic framework.

In [ ]:
# Core constants
D = 149.9992314000      # Donte constant - fixed-point attractor
I3 = 6.4939394023       # Normalization constant
S = D / I3              # Scaling ratio
LAMBDA = 0.16905        # Lightfoot constant - exponential decay rate
TAU = 1.0 / LAMBDA      # Time constant (seconds)

# Display constants
print("═" * 60)
print("PRIMAL LOGIC CONSTANTS")
print("═" * 60)
print(f"D (Donte constant):       {D:.10f}")
print(f"I3 (Normalization):       {I3:.10f}")
print(f"S (Scaling ratio):        {S:.10f}")
print(f"λ (Lightfoot constant):   {LAMBDA:.5f} s⁻¹")
print(f"τ (Time constant):        {TAU:.2f} seconds")
print("═" * 60)
print(f"\nPhysical meaning: System 'forgets' ~63% of past state every {TAU:.2f} seconds")

## Control Law Implementation

The Primal Logic control law:

$$\frac{d\psi}{dt} = -\lambda \cdot \psi(t) + K_E \cdot e(t)$$

where:
- ψ(t) = control command signal
- λ = exponential decay rate (Lightfoot constant)
- K_E = proportional error gain
- e(t) = tracking error

In [ ]:
def primal_logic_control(t_max=100.0, dt=0.01, KE=0.3, psi_0=1.0, noise_level=0.0):
    """
    Simulate Primal Logic control law.
    
    Parameters:
    -----------
    t_max : float
        Maximum simulation time (seconds)
    dt : float
        Time step (seconds)
    KE : float
        Error gain
    psi_0 : float
        Initial control command
    noise_level : float
        Gaussian noise standard deviation
    
    Returns:
    --------
    DataFrame with columns: t, psi, gamma, Ec
    """
    # Time array
    t = np.arange(0, t_max, dt)
    n = len(t)
    
    # State arrays
    psi = np.zeros(n)      # Control command
    gamma = np.zeros(n)    # Error signal
    Ec = np.zeros(n)       # Control energy
    
    # Initial conditions
    psi[0] = psi_0
    gamma[0] = 0.01 * psi_0  # Small initial error
    Ec[0] = 0.0
    
    # Simulation loop
    for i in range(1, n):
        # Add noise to error signal
        noise = np.random.normal(0, noise_level) if noise_level > 0 else 0
        
        # Error dynamics (simplified - error decays naturally)
        gamma[i] = gamma[i-1] * np.exp(-0.1 * dt) + noise
        
        # Control law: dψ/dt = -λ·ψ + KE·error
        dpsi_dt = -LAMBDA * psi[i-1] + KE * gamma[i]
        psi[i] = psi[i-1] + dpsi_dt * dt
        
        # Integrate control energy: Ec = ∫ ψ·γ dt
        Ec[i] = Ec[i-1] + psi[i] * gamma[i] * dt
    
    # Return as DataFrame
    return pd.DataFrame({
        't': t,
        'psi': psi,
        'gamma': gamma,
        'Ec': Ec
    })

print("✓ Control law implemented")

## Interactive Simulation

Run a simulation with default parameters and visualize the results.

In [ ]:
# Run simulation
data = primal_logic_control(t_max=50.0, dt=0.01, KE=0.3)

# Create interactive plot with Plotly
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Control Command ψ(t)', 'Error Signal γ(t)', 'Control Energy Ec(t)'),
    vertical_spacing=0.1
)

# ψ(t)
fig.add_trace(
    go.Scatter(x=data['t'], y=data['psi'], name='ψ(t)', line=dict(color='blue', width=2)),
    row=1, col=1
)

# γ(t)
fig.add_trace(
    go.Scatter(x=data['t'], y=data['gamma'], name='γ(t)', line=dict(color='red', width=2)),
    row=2, col=1
)

# Ec(t)
fig.add_trace(
    go.Scatter(x=data['t'], y=data['Ec'], name='Ec(t)', line=dict(color='green', width=2)),
    row=3, col=1
)

# Update layout
fig.update_xaxes(title_text="Time (s)", row=3, col=1)
fig.update_yaxes(title_text="ψ", row=1, col=1)
fig.update_yaxes(title_text="γ", row=2, col=1)
fig.update_yaxes(title_text="Ec", row=3, col=1)

fig.update_layout(
    height=800,
    title_text="Primal Logic Control Dynamics",
    showlegend=False
)

fig.show()

print(f"\nSimulation complete:")
print(f"  Final ψ:  {data['psi'].iloc[-1]:.6f}")
print(f"  Final γ:  {data['gamma'].iloc[-1]:.6f}")
print(f"  Final Ec: {data['Ec'].iloc[-1]:.6f}")
print(f"  Max |ψ|:  {np.max(np.abs(data['psi'])):.6f}")
print(f"  Max |Ec|: {np.max(np.abs(data['Ec'])):.6f}")

## Parameter Sweep: Effect of KE

Explore how the error gain KE affects system behavior.

In [ ]:
# Parameter sweep
KE_values = [0.0, 0.3, 0.5, 0.8]
colors = px.colors.qualitative.Set1

fig = go.Figure()

for i, KE in enumerate(KE_values):
    data = primal_logic_control(t_max=50.0, dt=0.01, KE=KE)
    fig.add_trace(
        go.Scatter(
            x=data['t'],
            y=data['psi'],
            name=f'KE = {KE}',
            line=dict(color=colors[i], width=2)
        )
    )

fig.update_layout(
    title='Effect of Error Gain KE on Control Command',
    xaxis_title='Time (s)',
    yaxis_title='ψ(t)',
    height=500,
    hovermode='x unified'
)

fig.show()

## Load Real Benchmark Data

Load and visualize actual benchmark run data from the repository.

In [ ]:
try:
    # Try to load benchmark data
    benchmark_files = [
        'run_default.csv',
        'run_ke05.csv',
        'run_mu02.csv'
    ]
    
    fig = go.Figure()
    
    for filename in benchmark_files:
        if os.path.exists(filename):
            # Read CSV (skip comment lines)
            df = pd.read_csv(filename, comment='#')
            
            # Extract parameters from filename
            label = filename.replace('run_', '').replace('.csv', '')
            
            fig.add_trace(
                go.Scatter(
                    x=df['t'],
                    y=df['psi'],
                    name=label,
                    mode='lines'
                )
            )
    
    fig.update_layout(
        title='Benchmark Run Comparison',
        xaxis_title='Time (s)',
        yaxis_title='ψ(t)',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()
    print("✓ Benchmark data loaded and visualized")
    
except Exception as e:
    print(f"Note: Benchmark files not found or error loading: {e}")
    print("This is expected if running on Colab without downloading data files.")

## Stability Analysis

Verify that the control energy Ec remains bounded (Lipschitz contractivity).

In [ ]:
# Run longer simulation to verify boundedness
data_long = primal_logic_control(t_max=200.0, dt=0.01, KE=0.5, noise_level=0.001)

# Check boundedness
max_psi = np.max(np.abs(data_long['psi']))
max_Ec = np.max(np.abs(data_long['Ec']))

print("="*60)
print("STABILITY ANALYSIS (200s simulation with noise)")
print("="*60)
print(f"Max |ψ|:  {max_psi:.6f}")
print(f"Max |Ec|: {max_Ec:.6f}")
print(f"\nBoundedness check:")
print(f"  ψ bounded:  {'✓ YES' if max_psi < 100 else '✗ NO'}")
print(f"  Ec bounded: {'✓ YES' if max_Ec < 1000 else '✗ NO'}")
print("="*60)

# Plot Ec over time
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data_long['t'], y=data_long['Ec'], name='Ec(t)', line=dict(color='green', width=2))
)
fig.update_layout(
    title='Control Energy Boundedness (Long-term)',
    xaxis_title='Time (s)',
    yaxis_title='Ec(t)',
    height=400
)
fig.show()

## Next Steps

- **02_nasa_data_visualization.ipynb**: Explore NASA mission data with interactive 3D visualizations
- **03_parameter_analysis.ipynb**: Advanced parameter sweeps and optimization
- **04_gpu_acceleration.ipynb**: GPU-accelerated simulations (Brev only)

## Learn More

- [Primal Logic Framework](../PRIMAL_LOGIC_FRAMEWORK.md)
- [Complete Setup Guide](../SETUP.md)
- [GitHub Repository](https://github.com/STLNFTART/MotorHandPro)

---

**Patent Pending:** U.S. Provisional Patent Application No. 63/842,846  
© 2025 Donte Lightfoot - The Phoney Express LLC / Locked In Safety